In [1]:
import numpy as np
import pyvista as pv
import os
from scipy import interpolate
import matplotlib.pyplot as plt

In [2]:
def create_base_streamlines(mesh):
    w,v,u = mesh.point_data['w'], mesh.point_data['v'], mesh.point_data['u']

    vectors = np.empty((mesh.n_points, 3))
    vectors[:,0] = u
    vectors[:,1] = v
    vectors[:,2] = w

    mesh['vectors'] = vectors

    streamlines, seed_pts = mesh.streamlines(
        'vectors', 
        return_source=True,
        n_points=100,
        source_radius=1.5,
        source_center=(6.0,0,0), 
        # pointa=(-0.5, -1.5, -0.5),
        # pointb=(-0.5, 1.5, 0.5)
    )
    
    return streamlines, seed_pts

In [3]:
def compute_downsampled_streamlines(s, seeds, base_mesh):
    '''
        Computes streamlines from seed points and down sample size
    '''
    w,v,u = base_mesh.point_data['w'], base_mesh.point_data['v'], base_mesh.point_data['u']

    u.shape = v.shape = w.shape = base_mesh.dimensions


    # Down Sample / Decimate
    u = u[::s, ::s, ::s] 
    v = v[::s, ::s, ::s]
    w = w[::s, ::s, ::s]

    dims = u.shape

    u = u.flatten()
    v = v.flatten()
    w = w.flatten()

    vectors = np.empty((int(len(u)), 3))
    vectors[:,0] = u
    vectors[:,1] = v
    vectors[:,2] = w

    mesh = pv.UniformGrid(
        dims=dims,
        spacing=tuple([x*s for x in list(base_mesh.spacing)]),
        origin=base_mesh.origin
    )

    mesh['vectors'] = vectors
    
    streamlines = mesh.streamlines_from_source(
        vectors='vectors', 
        source=seeds
    )
    
    return streamlines, mesh

In [4]:
def entry_point(filename):
    mesh = pv.read(filename)
    s, seed_pts = create_base_streamlines(mesh)
    s2, m2 = compute_downsampled_streamlines(2, seed_pts, mesh)
    s4, m4 = compute_downsampled_streamlines(4, seed_pts, mesh)
    s8, m8 = compute_downsampled_streamlines(8, seed_pts, mesh)
    s16, m16 = compute_downsampled_streamlines(16, seed_pts, mesh)
    s32, m32 = compute_downsampled_streamlines(32, seed_pts, mesh)
    s64, m64 = compute_downsampled_streamlines(64, seed_pts, mesh)
    return (s, s2, s4, s8, s16, s32, s64), (mesh, m2, m4, m8, m16, m32, m64)

In [5]:
reynolds160 = '.\\Data\\Structured160\\halfcylinder-2.00.vti'
# reynolds640 = '.\\Data\\Structured640\\halfcylinder-0.00.vti'
(s, s2, s4, s8, s16, s32, s64), (m, m2, m4, m8, m16, m32, m64) = entry_point(reynolds160)
# k, k2, k4, k8, k16, k32, k64 = entry_point(reynolds640)

In [6]:
p = pv.Plotter()

def plot_streamlines(value):
    error = int(value)
    streamlines = s
    mesh = m
    if error >= 2 and error < 4:
        streamlines = s2
        mesh = m2
    elif error >= 4 and error < 8:
        streamlines = s4
        mesh = m4
    elif error >= 8 and error < 16:
        streamlines = s8
        mesh = m8
    elif error >= 16 and error < 32:
        streamlines = s16
        mesh = m16
    elif error >= 32 and error < 64:
        streamlines = s32
        mesh = m32
    elif error >= 64:
        streamlines = s64                    
        mesh = m64
    
    boundary = mesh.decimate_boundary().extract_all_edges()
    # p.add_mesh(boundary, color='grey', opacity=.75)
    p.add_mesh(streamlines.tube(radius=0.02))
    

In [7]:
p.add_slider_widget(plot_streamlines, [0, 64], title='Downsample Size')
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)